In [25]:
# Modules
import xml.etree.ElementTree as ET
import random
import os
import glob
import pandas as pd
import nltk
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
# Local
import parsing_xml as px

In [5]:
exml = ET.parse('tests/latexmled_files/math.0407523.xml')

In [12]:
#prepare the dataset
allData = pd.DataFrame()
with open('data/out_defs.txt','r') as f1:
    all_data_texts = f1.readlines()
all_data_labels = len(all_data_texts)*[1.0]
with open('data/out_rand.txt', 'r') as f2:
    all_data_texts_rand = f2.readlines()
all_data_texts += all_data_texts_rand
all_data_labels += len(all_data_texts_rand)*[0.0]

# 1.0 will represent definitions is true 0.0 means it is false (not a definition)
allData['labels'] = all_data_labels
allData['texts'] = all_data_texts

In [18]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(allData['texts'], allData['labels'])

In [63]:
train_x.head(25)

168    there is a homomorphism           which takes ...
183    laboratoire j.–a. dieudonné,   université de n...
0             une orbifolde pure est un espace analyt...
16     the functor             is the covariant funct...
75     the functor                                   ...
28              [ in the special case that ] has a ge...
82            let  be a subscheme of . then a resolut...
101    given a vector bundle         on , a sub-bundl...
150                                                   \n
237           soit  une surface  lisse, et  un divise...
208    in the next paragraph, we just need that      ...
177                                                   \n
203    we keep the same notation as in the proof of t...
40     fix         and a -graded -module . the euler–...
130    by morita equivalence,         is isomorphic t...
94     a semi-stable curve           is called stable...
115    let           be a semisimple conjugacy class ...
163    this work is devoted to 

In [40]:
count_vect = CountVectorizer(analyzer='word', tokenizer=nltk.word_tokenize, ngram_range=(1,2))
count_vect.fit(allData['texts'])
xtrain = count_vect.transform(train_x)
xtest = count_vect.transform(test_x)

In [41]:
clf = naive_bayes.MultinomialNB().fit(xtrain, train_y)
predictions = clf.predict(xtest)
print(metrics.classification_report(predictions,test_y))

             precision    recall  f1-score   support

        0.0       0.61      0.87      0.71        23
        1.0       0.90      0.67      0.76        39

avg / total       0.79      0.74      0.75        62



In [42]:
print(metrics.confusion_matrix(predictions,test_y))

[[20  3]
 [13 26]]


In [50]:
26/29.

0.896551724137931

In [54]:
out_file = open('../out_rand.txt','a')
ns = {'latexml': 'http://dlmf.nist.gov/LaTeXML' }
for f in glob.glob('../2004_ends_with_3/*/*.xml'):
    try:
        exml = ET.parse(f)
        para_lst_nonrand = exml.findall('.//latexml:para',ns)
        para_lst = random.sample(para_lst_nonrand, 2)
        for p in para_lst:
            out_file.write(px.recutext1(p) + "\n")
    except ET.ParseError:
        pass
    except ValueError:
        print('article %s has few paragraphs'%f)
out_file.close()

article ../2004_ends_with_3/math.0412073/psfrag.xml has few paragraphs
article ../2004_ends_with_3/math.0412073/bibliography.xml has few paragraphs
article ../2004_ends_with_3/math.0412073/preamble.xml has few paragraphs
article ../2004_ends_with_3/math.0412533/gtoutput.xml has few paragraphs
